In [6]:
# # Pronóstico de Dengue con LSTM y Optimización de Hiperparámetros
# 
# Este notebook implementa un modelo LSTM para pronosticar casos de dengue usando PyTorch y Optuna para la optimización de hiperparámetros.

# %% [markdown]
# ## 1. Configuración e Importación de Librerías

# %%
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import optuna
from datetime import datetime, timedelta
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

# Configurar el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Dispositivo utilizado: {device}')

Dispositivo utilizado: cuda


In [7]:
# %% [markdown]
# ## 2. Carga y Preparación de Datos

# %%
# Cargar datos
df = pd.read_parquet('../../Datos/df_train.parquet')
print(f"Forma del DataFrame: {df.shape}")
print(f"Columnas: {df.columns.tolist()}")
print(df.head())

# %%
# Crear columna de fecha usando formato ISO de semanas
df['fecha'] = pd.to_datetime(df['anio'].astype(str) + df['semana'].astype(str) + '1', format='%G%V%u')

# Establecer la fecha como índice y ordenar
df.set_index('fecha', inplace=True)
df.sort_index(inplace=True)

print(f"Rango de fechas: {df.index.min()} - {df.index.max()}")
print(f"Número de registros: {len(df)}")
print(f"Número de barrios únicos (id_bar): {df['id_bar'].nunique()}")

# %% [markdown]

Forma del DataFrame: (3680, 20)
Columnas: ['id', 'id_bar', 'anio', 'semana', 'ESTRATO', 'area_barrio', 'dengue', 'concentraciones', 'vivienda', 'equipesado', 'sumideros', 'maquina', 'lluvia_mean', 'lluvia_var', 'lluvia_max', 'lluvia_min', 'temperatura_mean', 'temperatura_var', 'temperatura_max', 'temperatura_min']
          id  id_bar  anio  semana  ESTRATO  area_barrio  dengue  \
0  4_2015_01       4  2015       1      3.0        0.560     0.0   
1  5_2015_01       5  2015       1      3.0        0.842     0.0   
2  3_2015_01       3  2015       1      1.0        0.781     0.0   
3  8_2015_01       8  2015       1      2.0        0.394     0.0   
4  9_2015_01       9  2015       1      2.0        0.292     0.0   

   concentraciones  vivienda  equipesado  sumideros  maquina  lluvia_mean  \
0              0.0       0.0         0.0        0.0      0.0     0.000651   
1              0.0       0.0         0.0        0.0      0.0     0.000651   
2              0.0       0.0         0.0    

In [8]:
# ## 3. Preprocesamiento e Ingeniería de Características

# %%
# División de datos
train_data = df[df['anio'] < 2021].copy()
val_data = df[df['anio'] == 2021].copy()

print(f"Datos de entrenamiento: {train_data.shape}")
print(f"Datos de validación: {val_data.shape}")

# %%
# Identificar columnas numéricas y categóricas
categorical_cols = ['id_bar', 'ESTRATO']
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['anio', 'semana', 'dengue']]
target_col = 'dengue'

print(f"Columnas categóricas: {categorical_cols}")
print(f"Columnas numéricas: {numerical_cols}")
print(f"Variable objetivo: {target_col}")

# %%
# Crear mapeos para variables categóricas
id_bar_mapping = {id_bar: idx for idx, id_bar in enumerate(df['id_bar'].unique())}
estrato_mapping = {estrato: idx for idx, estrato in enumerate(df['ESTRATO'].unique())}

# Aplicar mapeos
train_data['id_bar_idx'] = train_data['id_bar'].map(id_bar_mapping)
train_data['estrato_idx'] = train_data['ESTRATO'].map(estrato_mapping)
val_data['id_bar_idx'] = val_data['id_bar'].map(id_bar_mapping)
val_data['estrato_idx'] = val_data['ESTRATO'].map(estrato_mapping)

# Escalado de variables numéricas
scaler = StandardScaler()
train_data[numerical_cols + [target_col]] = scaler.fit_transform(train_data[numerical_cols + [target_col]])
val_data[numerical_cols + [target_col]] = scaler.transform(val_data[numerical_cols + [target_col]])

# %% [markdown]
# ### Creación de Secuencias

# %%
def create_sequences(data, window_size, id_bar_list):
    """
    Crea secuencias de ventanas deslizantes para cada id_bar
    """
    sequences = []
    targets = []
    
    for id_bar in id_bar_list:
        bar_data = data[data['id_bar'] == id_bar].sort_index()
        
        if len(bar_data) > window_size:
            for i in range(len(bar_data) - window_size):
                # Características numéricas
                seq_numerical = bar_data[numerical_cols + [target_col]].iloc[i:i+window_size].values
                
                # Características categóricas (solo necesitamos el último valor)
                id_bar_idx = bar_data['id_bar_idx'].iloc[i+window_size-1]
                estrato_idx = bar_data['estrato_idx'].iloc[i+window_size-1]
                
                # Target
                target = bar_data[target_col].iloc[i+window_size]
                
                sequences.append({
                    'numerical': seq_numerical,
                    'id_bar': id_bar_idx,
                    'estrato': estrato_idx
                })
                targets.append(target)
    
    return sequences, targets

# %% [markdown]

Datos de entrenamiento: (3150, 20)
Datos de validación: (530, 20)
Columnas categóricas: ['id_bar', 'ESTRATO']
Columnas numéricas: ['id', 'area_barrio', 'concentraciones', 'vivienda', 'equipesado', 'sumideros', 'maquina', 'lluvia_mean', 'lluvia_var', 'lluvia_max', 'lluvia_min', 'temperatura_mean', 'temperatura_var', 'temperatura_max', 'temperatura_min']
Variable objetivo: dengue


In [9]:


# ## 4. Definición del Modelo LSTM con PyTorch

# %%
class DengueDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        return {
            'numerical': torch.FloatTensor(seq['numerical']),
            'id_bar': torch.LongTensor([seq['id_bar']]),
            'estrato': torch.LongTensor([seq['estrato']])
        }, torch.FloatTensor([self.targets[idx]])

# %%
class DengueLSTM(nn.Module):
    def __init__(self, n_numerical_features, n_id_bar, n_estrato, 
                 embedding_dim_bar=10, embedding_dim_estrato=3,
                 hidden_size=64, num_layers=2, dropout_rate=0.2):
        super(DengueLSTM, self).__init__()
        
        # Embeddings para variables categóricas
        self.embedding_id_bar = nn.Embedding(n_id_bar, embedding_dim_bar)
        self.embedding_estrato = nn.Embedding(n_estrato, embedding_dim_estrato)
        
        # LSTM
        self.lstm_input_size = n_numerical_features + embedding_dim_bar + embedding_dim_estrato
        self.lstm = nn.LSTM(self.lstm_input_size, hidden_size, 
                           num_layers, batch_first=True, dropout=dropout_rate)
        
        # Capa de salida
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, numerical, id_bar, estrato):
        batch_size, seq_len, _ = numerical.shape
        
        # Obtener embeddings
        emb_bar = self.embedding_id_bar(id_bar).squeeze(1)  # (batch, embedding_dim)
        emb_estrato = self.embedding_estrato(estrato).squeeze(1)  # (batch, embedding_dim)
        
        # Expandir embeddings para todas las timesteps
        emb_bar = emb_bar.unsqueeze(1).expand(-1, seq_len, -1)
        emb_estrato = emb_estrato.unsqueeze(1).expand(-1, seq_len, -1)
        
        # Concatenar todas las características
        x = torch.cat([numerical, emb_bar, emb_estrato], dim=2)
        
        # LSTM
        lstm_out, _ = self.lstm(x)
        
        # Usar solo la última salida
        last_output = lstm_out[:, -1, :]
        
        # Capa de salida
        out = self.dropout(last_output)
        out = self.fc(out)
        
        return out

# %% [markdown]

In [10]:

# ## 5. Optimización de Hiperparámetros con Optuna

# %%
def train_model(model, train_loader, val_loader, learning_rate, n_epochs, device):
    """
    Entrena el modelo y retorna el mejor MSE de validación
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    
    for epoch in range(n_epochs):
        # Entrenamiento
        model.train()
        train_loss = 0
        for batch in train_loader:
            inputs, targets = batch
            numerical = inputs['numerical'].to(device)
            id_bar = inputs['id_bar'].to(device)
            estrato = inputs['estrato'].to(device)
            targets = targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(numerical, id_bar, estrato)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validación
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                numerical = inputs['numerical'].to(device)
                id_bar = inputs['id_bar'].to(device)
                estrato = inputs['estrato'].to(device)
                targets = targets.to(device)
                
                outputs = model(numerical, id_bar, estrato)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
    
    return best_val_loss, train_losses, val_losses

# %%
def objective(trial):
    """
    Función objetivo para Optuna
    """
    # Hiperparámetros a optimizar
    window_size = trial.suggest_int('window_size', 8, 52, step=4)
    hidden_size = trial.suggest_categorical('hidden_size', [16 ,32, 64, 128])
    num_layers = trial.suggest_categorical('num_layers', [1, 2, 3])
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    
    # Crear secuencias
    id_bar_list = train_data['id_bar'].unique()
    train_sequences, train_targets = create_sequences(train_data, window_size, id_bar_list)
    val_sequences, val_targets = create_sequences(val_data, window_size, id_bar_list)
    
    # Crear datasets y dataloaders
    train_dataset = DengueDataset(train_sequences, train_targets)
    val_dataset = DengueDataset(val_sequences, val_targets)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Crear modelo
    n_numerical_features = len(numerical_cols) + 1  # +1 por la variable objetivo
    n_id_bar = len(id_bar_mapping)
    n_estrato = len(estrato_mapping)
    
    model = DengueLSTM(
        n_numerical_features=n_numerical_features,
        n_id_bar=n_id_bar,
        n_estrato=n_estrato,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout_rate=dropout_rate
    ).to(device)
    
    # Entrenar modelo
    best_val_loss, _, _ = train_model(model, train_loader, val_loader, 
                                      learning_rate, n_epochs=100, device=device)
    
    return best_val_loss

# %%
# Ejecutar estudio de Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

print(f"Mejor valor: {study.best_value}")
print(f"Mejores parámetros: {study.best_params}")

# %% [markdown]

[I 2025-06-23 15:00:06,390] A new study created in memory with name: no-name-d96bd88a-1fc5-422b-8486-35db555aab96
[I 2025-06-23 15:00:48,554] Trial 0 finished with value: 0.11192763488118847 and parameters: {'window_size': 16, 'hidden_size': 128, 'num_layers': 2, 'dropout_rate': 0.10611632451176703, 'learning_rate': 0.005453392633990735, 'batch_size': 16}. Best is trial 0 with value: 0.11192763488118847.
[I 2025-06-23 15:01:11,535] Trial 1 finished with value: 0.12078013519446056 and parameters: {'window_size': 16, 'hidden_size': 32, 'num_layers': 2, 'dropout_rate': 0.49223342138274395, 'learning_rate': 0.0010187342448629674, 'batch_size': 64}. Best is trial 0 with value: 0.11192763488118847.
[I 2025-06-23 15:01:56,617] Trial 2 finished with value: 0.08618915246592627 and parameters: {'window_size': 40, 'hidden_size': 16, 'num_layers': 2, 'dropout_rate': 0.24925671658713178, 'learning_rate': 0.00044775724040466776, 'batch_size': 16}. Best is trial 2 with value: 0.08618915246592627.
[I 

Mejor valor: 0.010852261446416378
Mejores parámetros: {'window_size': 52, 'hidden_size': 128, 'num_layers': 3, 'dropout_rate': 0.3105194523797959, 'learning_rate': 0.0023320421352418094, 'batch_size': 64}


In [11]:
# ## 6. Entrenamiento del Modelo Final

# %%
# Obtener mejores hiperparámetros
best_params = study.best_params
print(f"Mejores hiperparámetros encontrados:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

# %%
# Combinar datos de entrenamiento y validación
all_train_data = pd.concat([train_data, val_data])
print(f"Datos combinados para entrenamiento final: {all_train_data.shape}")

# Re-escalar con todos los datos
scaler_final = StandardScaler()
all_train_data_scaled = all_train_data.copy()
all_train_data_scaled[numerical_cols + [target_col]] = scaler_final.fit_transform(
    all_train_data[numerical_cols + [target_col]]
)

# %%
# Crear secuencias finales
id_bar_list = all_train_data['id_bar'].unique()
final_sequences, final_targets = create_sequences(
    all_train_data_scaled, 
    best_params['window_size'], 
    id_bar_list
)

# Crear dataset y dataloader final
final_dataset = DengueDataset(final_sequences, final_targets)
final_loader = DataLoader(
    final_dataset, 
    batch_size=best_params['batch_size'], 
    shuffle=True
)

# %%
# Crear y entrenar modelo final
n_numerical_features = len(numerical_cols) + 1
n_id_bar = len(id_bar_mapping)
n_estrato = len(estrato_mapping)

final_model = DengueLSTM(
    n_numerical_features=n_numerical_features,
    n_id_bar=n_id_bar,
    n_estrato=n_estrato,
    hidden_size=best_params['hidden_size'],
    num_layers=best_params['num_layers'],
    dropout_rate=best_params['dropout_rate']
).to(device)

# Entrenar modelo final
criterion = nn.MSELoss()
optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])

train_losses = []
n_epochs_final = 300

for epoch in range(n_epochs_final):
    final_model.train()
    epoch_loss = 0
    
    for batch in final_loader:
        inputs, targets = batch
        numerical = inputs['numerical'].to(device)
        id_bar = inputs['id_bar'].to(device)
        estrato = inputs['estrato'].to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        outputs = final_model(numerical, id_bar, estrato)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(final_loader)
    train_losses.append(avg_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Época {epoch + 1}/{n_epochs_final}, Pérdida: {avg_loss:.4f}")

# %%
# Visualizar curva de pérdida
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(1, n_epochs_final + 1)),
    y=train_losses,
    mode='lines',
    name='Pérdida de entrenamiento',
    line=dict(color='blue', width=2)
))

fig.update_layout(
    title='Curva de Pérdida durante el Entrenamiento',
    xaxis_title='Época',
    yaxis_title='MSE',
    template='plotly_white',
    height=500
)

fig.show()

# %% [markdown]

Mejores hiperparámetros encontrados:
  window_size: 52
  hidden_size: 128
  num_layers: 3
  dropout_rate: 0.3105194523797959
  learning_rate: 0.0023320421352418094
  batch_size: 64
Datos combinados para entrenamiento final: (3680, 22)
Época 10/300, Pérdida: 0.2610
Época 20/300, Pérdida: 0.1840
Época 30/300, Pérdida: 0.1195
Época 40/300, Pérdida: 0.1010
Época 50/300, Pérdida: 0.0785
Época 60/300, Pérdida: 0.0614
Época 70/300, Pérdida: 0.0507
Época 80/300, Pérdida: 0.0359
Época 90/300, Pérdida: 0.0352
Época 100/300, Pérdida: 0.0302
Época 110/300, Pérdida: 0.0292
Época 120/300, Pérdida: 0.0261
Época 130/300, Pérdida: 0.0216
Época 140/300, Pérdida: 0.0189
Época 150/300, Pérdida: 0.0185
Época 160/300, Pérdida: 0.0192
Época 170/300, Pérdida: 0.0181
Época 180/300, Pérdida: 0.0175
Época 190/300, Pérdida: 0.0164
Época 200/300, Pérdida: 0.0149
Época 210/300, Pérdida: 0.0164
Época 220/300, Pérdida: 0.0140
Época 230/300, Pérdida: 0.0155
Época 240/300, Pérdida: 0.0124
Época 250/300, Pérdida: 0.0147

In [12]:


# ## 7. Generación de Pronósticos para 2022

# %%
def generate_predictions_2022(model, data, window_size, scaler, device):
    """
    Genera predicciones autorregresivas para 2022
    """
    model.eval()
    predictions_2022 = []
    
    # Obtener todas las semanas de 2022
    start_date_2022 = pd.to_datetime('2022-01-03')  # Primer lunes de 2022
    weeks_2022 = []
    for week in range(52):
        week_date = start_date_2022 + timedelta(weeks=week)
        year_week = week_date.isocalendar()
        weeks_2022.append((year_week.year, year_week.week))
    
    for id_bar in data['id_bar'].unique():
        bar_data = data[data['id_bar'] == id_bar].sort_index()
        
        # Tomar las últimas 'window_size' semanas
        last_sequence = bar_data.tail(window_size).copy()
        
        # Para cada semana de 2022
        for year, week in weeks_2022:
            # Preparar entrada
            numerical_features = last_sequence[numerical_cols + [target_col]].values
            numerical_tensor = torch.FloatTensor(numerical_features).unsqueeze(0).to(device)
            
            id_bar_idx = bar_data['id_bar_idx'].iloc[-1]
            estrato_idx = bar_data['estrato_idx'].iloc[-1]
            id_bar_tensor = torch.LongTensor([id_bar_idx]).unsqueeze(0).to(device)
            estrato_tensor = torch.LongTensor([estrato_idx]).unsqueeze(0).to(device)
            
            # Hacer predicción
            with torch.no_grad():
                prediction_scaled = final_model(numerical_tensor, id_bar_tensor, estrato_tensor)
                prediction_scaled = prediction_scaled.cpu().numpy()[0, 0]
            
            # Des-escalar predicción
            # Crear array para des-escalar solo la columna de dengue
            dummy_array = np.zeros((1, len(numerical_cols) + 1))
            dummy_array[0, -1] = prediction_scaled
            prediction_original = scaler.inverse_transform(dummy_array)[0, -1]
            
            # Guardar predicción
            predictions_2022.append({
                'id_bar': id_bar,
                'anio': year,
                'semana': week,
                'dengue': max(0, prediction_original)  # Asegurar no negativos
            })
            
            # Actualizar secuencia para próxima predicción
            # Crear nueva fila con la predicción
            new_row = last_sequence.iloc[-1:].copy()
            new_row[target_col] = prediction_scaled
            
            # Deslizar ventana
            last_sequence = pd.concat([last_sequence.iloc[1:], new_row])
    
    return pd.DataFrame(predictions_2022)

# %%
# Generar predicciones
print("Generando pronósticos para 2022...")
predictions_df = generate_predictions_2022(
    final_model, 
    all_train_data_scaled, 
    best_params['window_size'], 
    scaler_final, 
    device
)

print(f"Pronósticos generados: {predictions_df.shape}")
print(predictions_df.head())

# %% [markdown]

Generando pronósticos para 2022...
Pronósticos generados: (520, 4)
   id_bar  anio  semana  dengue
0       4  2022       1     0.0
1       4  2022       2     0.0
2       4  2022       3     0.0
3       4  2022       4     0.0
4       4  2022       5     0.0


In [13]:

# ## 8. Creación del Archivo de Salida

# %%
# Crear columna 'id' con el formato requerido
predictions_df['semana_str'] = predictions_df['semana'].apply(lambda x: f"{x:02d}")
predictions_df['id'] = predictions_df['id_bar'].astype(str) + '_' + \
                       predictions_df['anio'].astype(str) + '_' + \
                       predictions_df['semana_str']

# Seleccionar columnas finales
output_df = predictions_df[['id', 'dengue']]

# Verificar formato
print(f"Formato del archivo de salida:")
print(output_df.head(10))
print(f"\nTotal de predicciones: {len(output_df)}")
print(f"Predicciones por barrio: {len(output_df) / predictions_df['id_bar'].nunique()}")

# %%
# Guardar archivo
fecha_actual = datetime.now().strftime("%Y%m%d")
output_df.to_csv(f'pronosticos_LSTM_C_{fecha_actual}.csv', index=False)
print("Archivo 'pronosticos_dengue_2022.csv' guardado exitosamente!")

# %%
# Visualización de algunas predicciones
sample_bars = predictions_df['id_bar'].unique()[:5]
fig = make_subplots(rows=3, cols=2, subplot_titles=[f'Barrio {bar}' for bar in sample_bars[:6]])

for idx, id_bar in enumerate(sample_bars[:6]):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    bar_predictions = predictions_df[predictions_df['id_bar'] == id_bar]
    
    fig.add_trace(
        go.Scatter(
            x=bar_predictions['semana'],
            y=bar_predictions['dengue'],
            mode='lines+markers',
            name=f'Barrio {id_bar}'
        ),
        row=row, col=col
    )

fig.update_layout(
    title='Pronósticos de Dengue para 2022 - Muestra de Barrios',
    height=800,
    showlegend=False
)

fig.show()

# %%
# Estadísticas finales
print("\nEstadísticas de los pronósticos:")
print(f"Promedio de casos pronosticados: {output_df['dengue'].mean():.2f}")
print(f"Desviación estándar: {output_df['dengue'].std():.2f}")
print(f"Mínimo: {output_df['dengue'].min():.2f}")
print(f"Máximo: {output_df['dengue'].max():.2f}")
print(f"Mediana: {output_df['dengue'].median():.2f}")

Formato del archivo de salida:
          id    dengue
0  4_2022_01  0.000000
1  4_2022_02  0.000000
2  4_2022_03  0.000000
3  4_2022_04  0.000000
4  4_2022_05  0.000000
5  4_2022_06  0.074723
6  4_2022_07  0.000000
7  4_2022_08  0.000000
8  4_2022_09  0.000000
9  4_2022_10  0.000000

Total de predicciones: 520
Predicciones por barrio: 52.0
Archivo 'pronosticos_dengue_2022.csv' guardado exitosamente!



Estadísticas de los pronósticos:
Promedio de casos pronosticados: 0.00
Desviación estándar: 0.02
Mínimo: 0.00
Máximo: 0.22
Mediana: 0.00
